# Extracting timeseries and calculating the functional connectome (partial correlation) of every participant and group

## Importing all necessary modules and load the dataset

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bids import BIDSLayout

from nilearn import plotting, image
from nilearn.image import index_img
from nilearn.plotting import find_xyz_cut_coords
from nilearn.connectome import ConnectivityMeasure
from nilearn.maskers import NiftiMapsMasker
from nilearn.interfaces.fmriprep import load_confounds, load_confounds_strategy

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# load all the BIDS-compatible data with BIDSLayout
fmriprep_dir = '/media/maria/Elements/Bipolife_BIDS/A1/derivatives/fmriprep'
layout = BIDSLayout(fmriprep_dir, validate=False) # validation = false cause fmriprep ouputs are just BIDS-like

# get subject list
sub_idx = layout.get_subjects()

/home/maria/miniconda3/lib/python3.8/site-packages/bids/layout/models.py:148: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  warnings.warn("The 'extension' entity currently excludes the leading dot ('.'). "


In [3]:
len(sub_idx) # should be 402
#sub_idx

402

## Creating the different group lists based on the excel table

In [4]:
# load the excel sheet and make sure that it is in the right format so that pandas can access it, xlrd extension is needed
excelfile= '/media/maria/Elements/Bipolife_BIDS/mastertable_rename_convert_0903.xls' 
# access the corresponding sheets for every group
nr_df = pd.read_excel(excelfile, sheet_name="no_risk")
lr_df = pd.read_excel(excelfile, sheet_name="low_risk")
hr_df = pd.read_excel(excelfile, sheet_name="high_risk")
bd_df = pd.read_excel(excelfile, sheet_name="bipolar")
hc_df = pd.read_excel(excelfile, sheet_name="control")

In [5]:
# get a subject list for every group
sub_nr = nr_df['BIDS']
sub_lr = lr_df['BIDS']
sub_hr = hr_df['BIDS']
sub_bd = bd_df['BIDS']
sub_hc = hc_df['BIDS']

In [6]:
# list of excluded participants
excl_subs = ['BC15','MC01','TC06','BR05','MD06','HD39','MA05','BR04','FR01','FD05','MS01','DS02','BR30','GS01','HD15','HD35','HD25','TF01','TA09','FA13','BD17','MR06','HD22','TR01','MD16','FD15','DR15','BD20','FD22','TS02','DF04','HD37','DR13','BS09','OR05','HD40','BD25','TS01','HS03','DR21','GS02','TR04','TR05','TR06']

In [7]:
len(sub_idx)-len(excl_subs)

358

## Loading the mask and pass it to the NiftiMapsMasker

In [4]:
# load the image and labels 
all_regions_img = image.load_img('/media/maria/Elements/Bipolife_BIDS/A1/code/maria_thesis/dmn_limbic_2903.nii.gz')
all_region_labels = np.load('/media/maria/Elements/Bipolife_BIDS/A1/code/maria_thesis/dmn_limbic_labels_2903.npy', allow_pickle= True)

In [5]:
all_region_labels

array(['Ventromedial prefrontal cortex', 'Middle temporal gyrus RH',
       'Posterior cingulate cortex',
       'Dorsomedial prefrontal cortex antero-superior',
       'Middle temporal gyrus LH', 'Posterior cingulate cortex inferior',
       'Hippocampal fissure',
       'Dorsomedial prefrontal cortex anteromedial',
       'Parieto-occipital sulcus anterior', 'Anterior cingulate cortex',
       'Retrosplenial cortex', 'Superior frontal sulcus anterior LH',
       'Ventromedial prefrontal cortex anterior', 'Thalamus',
       'Superior frontal sulcus LH', 'Superior frontal sulcus RH',
       'Angular gyrus posterior LH', 'Angular gyrus posterior RH',
       'Parahippocampal gyrus LH', 'Parahippocampal gyrus RH',
       'Pars triangularis LH', 'Pars triangularis RH',
       'Superior frontal sulcus anterior LH2',
       'Superior frontal sulcus anterior RH', 'Angular gyrus inferior LH',
       'Angular gyrus inferior RH', 'Amygdala LH', 'Amygdala RH',
       'Hippocampus posterior LH', '

In [10]:
# create the mask for the DMN and limbic structures 
maps_masker = NiftiMapsMasker(maps_img = all_regions_img, 
                              smoothing_fwhm = 4.0, 
                              standardize = True,
                              standardize_confounds = True, 
                              detrend = True, 
                              low_pass= 0.08, 
                              high_pass= 0.01,
                              t_r = 2, #can be found in the metadata of the scans 
                              reports = True,
                              verbose=1)
maps_masker.fit()

[NiftiMapsMasker.fit] loading regions from Nifti1Image('/media/maria/Elements/Bipolife_BIDS/A1/code/maria_thesis/dmn_limbic_2903.nii.gz')


NiftiMapsMasker(detrend=True, high_pass=0.01, low_pass=0.08,
                maps_img=<nibabel.nifti1.Nifti1Image object at 0x7f6203864610>,
                smoothing_fwhm=4.0, standardize=True, t_r=2, verbose=1)

## Creating a loop to get the functional connectome of every subject 

In [14]:
ts_nr = []
ts_lr = []
ts_hr = []
ts_bd = []
ts_hc = []
ts_all = []

In [15]:
ts_all

[]

In [ ]:
# name the atlas you have choosen  
atlas = 'difumo'
# choose how many dummy scans should be dropped
tr_drop = 5

for kind in ['partial correlation']: # you can change or add other conectivity measurements here 
    
    kind_name = 'partial_correlation' # so that we do not have the space later on then we are naming our files
    
    for sub in sub_idx:
        
        if sub in excl_subs:
            continue

        # create a directory for the connectivity matrices
        # be aware A1 and A2 differ regarding their folder structure, A2 includes an extra pre-ses folder
        print('Creating a directory to save the computed correlation matrices')
        # access the right directory 
        dir_matrices_derivatives = f'/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-{kind_name}/sub-{sub}/'

        # if the direcetory does not exist it will be created 
        if not os.path.exists(dir_matrices_derivatives):
            os.makedirs(dir_matrices_derivatives)
            print(f'Created directory:{dir_matrices_derivatives}')
        else:
            print(f'Directory {dir_matrices_derivatives} already exists. None is created.')

        print(f'Loading the data from subject {sub}')
        # get the rsfMRI file paths
        sub_rest_file = layout.get(subject = sub, return_type= 'file', datatype='func', task = 'rest', space = 'MNI152NLin2009cAsym', suffix='bold', extension='.nii.gz')
        
        # prepare data for analysis
        # load the functional file and drop the dummy scans 
        func_img = image.load_img(sub_rest_file)
        func_img = func_img.slicer[:,:,:,tr_drop:]

        # load and select confound files
        confounds_simple = load_confounds(sub_rest_file,
                                          strategy=('motion','wm_csf','global_signal'),
                                          motion = 'full',
                                          wm_csf = 'full',
                                          global_signal = 'full')
        confounds = confounds_simple[0].loc[tr_drop:] 


        # extract timeseries and group them according to classification
        print(f'Extracting connectivity matrices of the partial correlation kind for {sub}')
        sub_time_series = maps_masker.transform_single_imgs(func_img, confounds=confounds)


        # create a list of all subject timeseries 
        ts_all.append(sub_time_series)

        # if the subject is in the no_risk group
        if sub in sub_nr.values:
            ts_nr.append(sub_time_series)
        # if the subject is in the low_risk group
        if sub in sub_lr.values:
            ts_lr.append(sub_time_series)
        # if the subject is in the high_risk group
        if sub in sub_hr.values:
            ts_hr.append(sub_time_series)
        # if the subject is in the bipolar disorder group
        if sub in sub_bd.values:
            ts_bd.append(sub_time_series)
        # if the subject is in the healthy control group
        if sub in sub_hc.values:
            ts_hc.append(sub_time_series)



        # get functional connectome of every participant
        # create a matrix of the functional connectivity values (for plotting)
        sub_connectivity_measure = ConnectivityMeasure(kind=kind)
        sub_correlation_matrix = sub_connectivity_measure.fit_transform([sub_time_series])[0]
        
        # create a vector of the functional connectivity values (for statistical analysis)
        sub_connectivity_vectorized = ConnectivityMeasure(kind=kind, vectorize = True, discard_diagonal = True)
        sub_correlation_vector = sub_connectivity_vectorized.fit_transform([sub_time_series])[0]
        np.save(f'{dir_matrices_derivatives}sub-{sub}_atlas-{atlas}_kind-{kind_name}_connectivity_vector', sub_correlation_vector)

        # save the matrices and vectors as csv files
        print(f'Saving dataframe for {sub} into a .csv file in : {dir_matrices_derivatives}')
        # matrices
        sub_connectivity_matrix = pd.DataFrame(data=sub_correlation_matrix, 
                                               index=all_region_labels[:], 
                                               columns=all_region_labels[:])
        sub_connectivity_matrix.to_csv(f'{dir_matrices_derivatives}sub-{sub}_atlas-{atlas}_kind-{kind_name}_connectivity_matrix.csv')
        # vectors
        sub_connectivity_vector = pd.DataFrame(data=sub_correlation_vector)
        sub_connectivity_vector.to_csv(f'{dir_matrices_derivatives}sub-{sub}_atlas-{atlas}_kind-{kind_name}_connectivity_vector.csv')

        #save the plot

        # get bigger labels in the matrix plot
        plt.rcParams.update({'font.size': 35})
        
        # not ordered
        print(f'Saving the plot for {sub} in : {dir_matrices_derivatives}')
        display1 = plotting.plot_matrix(sub_correlation_matrix, 
                                        figure=(30, 30), 
                                        labels=all_region_labels[:], 
                                        )
        display1.figure.savefig(f'{dir_matrices_derivatives}sub-{sub}_atlas-{atlas}_kind-{kind_name}_connectivity_matrix.png')  
        plt.close()

Creating a directory to save the computed correlation matrices
Directory /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BC01/ already exists. None is created.
Loading the data from subject BC01
Extracting connectivity matrices of the correlation kind for BC01
[NiftiMapsMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(60, 71, 55, 245),
affine=array([[   3.2809999,    0.       ,    0.       ,  -96.5      ],
       [   0.       ,    3.2809999,    0.       , -132.5      ],
       [   0.       ,    0.       ,    3.5999999,  -78.5      ],
       [   0.       ,    0.       ,    0.       ,    1.       ]])
)
[NiftiMapsMasker.transform_single_imgs] Smoothing images
[NiftiMapsMasker.transform_single_imgs] Extracting region signals
[NiftiMapsMasker.transform_single_imgs] Cleaning extracted signals
Saving dataframe for BC01 into a .csv file in : /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-c

[NiftiMapsMasker.transform_single_imgs] Extracting region signals
[NiftiMapsMasker.transform_single_imgs] Cleaning extracted signals
Saving dataframe for BC08 into a .csv file in : /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BC08/
Saving the plot for BC08 in : /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BC08/
Creating a directory to save the computed correlation matrices
Created directory:/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BC09/
Loading the data from subject BC09
Extracting connectivity matrices of the correlation kind for BC09
[NiftiMapsMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(60, 71, 55, 245),
affine=array([[   3.2809999,    0.       ,    0.       ,  -96.5      ],
       [   0.       ,    3.2809999,    0.       , -132.5      ],
       [   0.       ,    0.       ,    3.5999999,  -78.5   

Creating a directory to save the computed correlation matrices
Created directory:/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BC17/
Loading the data from subject BC17
Extracting connectivity matrices of the correlation kind for BC17
[NiftiMapsMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(60, 71, 55, 245),
affine=array([[   3.2809999,    0.       ,    0.       ,  -96.5      ],
       [   0.       ,    3.2809999,    0.       , -132.5      ],
       [   0.       ,    0.       ,    3.5999999,  -78.5      ],
       [   0.       ,    0.       ,    0.       ,    1.       ]])
)
[NiftiMapsMasker.transform_single_imgs] Smoothing images
[NiftiMapsMasker.transform_single_imgs] Extracting region signals
[NiftiMapsMasker.transform_single_imgs] Cleaning extracted signals
Saving dataframe for BC17 into a .csv file in : /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BC17/
Savi

[NiftiMapsMasker.transform_single_imgs] Extracting region signals
[NiftiMapsMasker.transform_single_imgs] Cleaning extracted signals
Saving dataframe for BD03 into a .csv file in : /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BD03/
Saving the plot for BD03 in : /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BD03/
Creating a directory to save the computed correlation matrices
Created directory:/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BD04/
Loading the data from subject BD04
Extracting connectivity matrices of the correlation kind for BD04
[NiftiMapsMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(60, 71, 55, 245),
affine=array([[   3.2809999,    0.       ,    0.       ,  -96.5      ],
       [   0.       ,    3.2809999,    0.       , -132.5      ],
       [   0.       ,    0.       ,    3.5999999,  -78.5   

Creating a directory to save the computed correlation matrices
Created directory:/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BD11/
Loading the data from subject BD11
Extracting connectivity matrices of the correlation kind for BD11
[NiftiMapsMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(60, 71, 55, 245),
affine=array([[   3.2809999,    0.       ,    0.       ,  -96.5      ],
       [   0.       ,    3.2809999,    0.       , -132.5      ],
       [   0.       ,    0.       ,    3.5999999,  -78.5      ],
       [   0.       ,    0.       ,    0.       ,    1.       ]])
)
[NiftiMapsMasker.transform_single_imgs] Smoothing images
[NiftiMapsMasker.transform_single_imgs] Extracting region signals
[NiftiMapsMasker.transform_single_imgs] Cleaning extracted signals
Saving dataframe for BD11 into a .csv file in : /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BD11/
Savi

...

Creating a directory to save the computed correlation matrices
Created directory:/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BF03/
Loading the data from subject BF03
Extracting connectivity matrices of the correlation kind for BF03
[NiftiMapsMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(60, 71, 55, 245),
affine=array([[   3.2809999,    0.       ,    0.       ,  -96.5      ],
       [   0.       ,    3.2809999,    0.       , -132.5      ],
       [   0.       ,    0.       ,    3.5999999,  -78.5      ],
       [   0.       ,    0.       ,    0.       ,    1.       ]])
)
[NiftiMapsMasker.transform_single_imgs] Smoothing images
[NiftiMapsMasker.transform_single_imgs] Extracting region signals
[NiftiMapsMasker.transform_single_imgs] Cleaning extracted signals
Saving dataframe for BF03 into a .csv file in : /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BF03/
Savi

[NiftiMapsMasker.transform_single_imgs] Extracting region signals
[NiftiMapsMasker.transform_single_imgs] Cleaning extracted signals
Saving dataframe for BF10 into a .csv file in : /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BF10/
Saving the plot for BF10 in : /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BF10/
Creating a directory to save the computed correlation matrices
Created directory:/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BF11/
Loading the data from subject BF11
Extracting connectivity matrices of the correlation kind for BF11
[NiftiMapsMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(60, 71, 55, 245),
affine=array([[   3.2809999,    0.       ,    0.       ,  -96.5      ],
       [   0.       ,    3.2809999,    0.       , -132.5      ],
       [   0.       ,    0.       ,    3.5999999,  -78.5   

Creating a directory to save the computed correlation matrices
Created directory:/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BR09/
Loading the data from subject BR09
Extracting connectivity matrices of the correlation kind for BR09
[NiftiMapsMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(60, 71, 55, 245),
affine=array([[   3.2809999,    0.       ,    0.       ,  -96.5      ],
       [   0.       ,    3.2809999,    0.       , -132.5      ],
       [   0.       ,    0.       ,    3.5999999,  -78.5      ],
       [   0.       ,    0.       ,    0.       ,    1.       ]])
)
[NiftiMapsMasker.transform_single_imgs] Smoothing images
[NiftiMapsMasker.transform_single_imgs] Extracting region signals
[NiftiMapsMasker.transform_single_imgs] Cleaning extracted signals
Saving dataframe for BR09 into a .csv file in : /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BR09/
Savi

[NiftiMapsMasker.transform_single_imgs] Extracting region signals
[NiftiMapsMasker.transform_single_imgs] Cleaning extracted signals
Saving dataframe for BR16 into a .csv file in : /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BR16/
Saving the plot for BR16 in : /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BR16/
Creating a directory to save the computed correlation matrices
Created directory:/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BR17/
Loading the data from subject BR17
Extracting connectivity matrices of the correlation kind for BR17
[NiftiMapsMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(60, 71, 55, 245),
affine=array([[   3.2809999,    0.       ,    0.       ,  -96.5      ],
       [   0.       ,    3.2809999,    0.       , -132.5      ],
       [   0.       ,    0.       ,    3.5999999,  -78.5   

Creating a directory to save the computed correlation matrices
Created directory:/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BR24/
Loading the data from subject BR24
Extracting connectivity matrices of the correlation kind for BR24
[NiftiMapsMasker.transform_single_imgs] Loading data from Nifti1Image(
shape=(60, 71, 55, 245),
affine=array([[   3.2809999,    0.       ,    0.       ,  -96.5      ],
       [   0.       ,    3.2809999,    0.       , -132.5      ],
       [   0.       ,    0.       ,    3.5999999,  -78.5      ],
       [   0.       ,    0.       ,    0.       ,    1.       ]])
)
[NiftiMapsMasker.transform_single_imgs] Smoothing images
[NiftiMapsMasker.transform_single_imgs] Extracting region signals
[NiftiMapsMasker.transform_single_imgs] Cleaning extracted signals
Saving dataframe for BR24 into a .csv file in : /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/subs/kind-correlation/sub-BR24/
Savi

## Creating group partial correlation matrices

In [10]:
import numpy as np
ts_hc = np.load('/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/groups/group_timeseries_hc_2903.npy')
ts_nr = np.load('/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/groups/group_timeseries_nr_2903.npy')
ts_lr = np.load('/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/groups/group_timeseries_lr_2903.npy')
ts_hr = np.load('/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/groups/group_timeseries_hr_2903.npy')
ts_bd = np.load('/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/groups/group_timeseries_bd_2903.npy')

In [6]:
# get the coords for every ROI for the connectome plot
coords = []
for i in range(len(all_region_labels)):
    cur_img = index_img(all_regions_img, i)
    coords.append(find_xyz_cut_coords(cur_img))
#len(coords)

In [11]:
# name the atlas you have choosen  
atlas = 'difumo'

for kind in ['partial correlation']: # you can change or add other conectivity measurements here 
    
    kind = 'partial_correlation' # so that we do not have the space later on then we are naming our files
    
    print(f'Calculation for {kind}')

    for group in [ts_nr, ts_lr, ts_hr, ts_hc, ts_bd]:

        #get the list name
        group_name = [name for name in globals() if globals()[name] is group][0]
        # get the real name and not the variable name
        group_name = group_name[slice(3,6)]

        print('Creating a directory to save the computed group correlation matrices')
        # access the right directory 
        dir_matrices_derivatives = f'/media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/groups/kind-{kind}/group-{group_name}/2903/'

        # if the direcetory does not exist it will be created 
        if not os.path.exists(dir_matrices_derivatives):
            os.makedirs(dir_matrices_derivatives)
            print(f'Created directory:{dir_matrices_derivatives}')
        else:
            print(f'Directory {dir_matrices_derivatives} already exists. None is created.')

        # create the connectivity matrix for a group
        group_connectivity_measure = ConnectivityMeasure(kind=kind)
        group_correlation_matrices = group_connectivity_measure.fit_transform(group)
        
        # we are interested in the mean correlation matrix of the group 
        mean_correlation_matrix = group_connectivity_measure.mean_

        # save the matrices as a csv file with region labels 
        group_connectivity_matrix_withlabels  = pd.DataFrame(data=mean_correlation_matrix, index=all_region_labels[:], columns=all_region_labels)
        group_connectivity_matrix_withlabels.to_csv(f'{dir_matrices_derivatives}group-{group_name}_atlas-{atlas}_kind-{kind}_matrix_labels.csv')

        # get bigger labels in the matrix plot
        plt.rcParams.update({'font.size': 35})
        
        # plot the matrices and save them 
        display1 = plotting.plot_matrix(mean_correlation_matrix, 
                                       figure=(30, 30), 
                                       labels=all_region_labels[:],
                                       vmin = -0.5,
                                       vmax = 0.5)
        display1.figure.savefig(f'{dir_matrices_derivatives}group-{group_name}_atlas-{atlas}_kind-{kind}_matrix.png')  
        plt.close()

        # plot the connectome and save it as png image 
        plotting.plot_connectome(mean_correlation_matrix,
                                 node_coords = coords,
                                 node_size = 30,
                                 edge_threshold='90%',
                                 edge_cmap = 'seismic',
                                 edge_vmin = -0.5,
                                 edge_vmax = 0.5,
                                 display_mode="lzr",
                                 colorbar=True,
                                 title=f'Connectome of {group_name}',
                                 output_file = f'{dir_matrices_derivatives}group-{group_name}_atlas-{atlas}_kind-{kind}_connectome.png')


Calculation for partial correlation
Creating a directory to save the computed group correlation matrices
Directory /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/groups/kind-partial correlation/group-nr/2903/ already exists. None is created.
Creating a directory to save the computed group correlation matrices
Directory /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/groups/kind-partial correlation/group-lr/2903/ already exists. None is created.
Creating a directory to save the computed group correlation matrices
Directory /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/groups/kind-partial correlation/group-hr/2903/ already exists. None is created.
Creating a directory to save the computed group correlation matrices
Directory /media/maria/Elements/Bipolife_BIDS/A1/derivatives/connectivity_matrices/groups/kind-partial correlation/group-hc/2903/ already exists. None is created.
Creating a directory to save the com